In [ ]:
%load_ext autoreload
import os
import sys
import h5py as h5
import numpy as np
import pandas as pd
import plotly.graph_objects as go

cwd = os.getcwd()
sys.path.append(os.path.dirname(cwd))

%autoreload 2
from zcu_tools import load_data  # noqa: E402
from zcu_tools.analysis.fluxdep import (  # noqa: E402
    InteractiveLines,
    InteractiveSelector,
    calculate_energy,
    energy2transition,
    preprocess_data,
    search_in_database,
    spectrum_analyze,
    fit_spectrum
)

In [ ]:
qub_name = "S001"

cflx = None
eflx = None
s_points = {}

In [ ]:
# load_path = "result/2DQ9/params.csv"
# spect = pd.read_csv(load_path)
# spect = spect[spect["qubit"] == qub_name]

# cflx = spect["cflx"].values[0]
# eflx = cflx + spect["Period"].values[0] / 2
# sp_params = (spect["EJ"].values[0], spect["EC"].values[0], spect["EL"].values[0])
# cflx, eflx, sp_params

# Load Spectrum

In [ ]:
spect_path = "../Database/S001/2025/02/Data_0222/S001_TwoTone_Flux_001.hdf5"
spectrum, fpts, flxs = load_data(spect_path, server_ip="100.105.52.95", port=4999)
flxs, fpts, spectrum = preprocess_data(flxs, fpts, spectrum)

s_spectrum = np.abs(spectrum - np.mean(spectrum, axis=0, keepdims=True))
s_spectrum /= np.std(s_spectrum, axis=0, keepdims=True)

In [ ]:
%matplotlib widget
actLine = InteractiveLines(s_spectrum**2, flxs, fpts, cflx, eflx)

In [ ]:
cflx, eflx = actLine.get_positions()
halfp = eflx - cflx
cflx, eflx, halfp

In [ ]:
flxs = (flxs - cflx) / (2 * halfp) + 0.5

In [ ]:
%matplotlib widget
s_flxs, s_fpts = spectrum_analyze(flxs, fpts, spectrum, 8.0)

actSel = InteractiveSelector(s_spectrum, flxs, fpts, s_flxs, s_fpts)

In [ ]:
s_flxs, s_fpts, _ = actSel.get_selected_points()

In [ ]:
name = spect_path.split("/")[-1]
s_points.update({name: {
    "cflx": cflx,
    "spectrum": (flxs, fpts, spectrum),
    "points": (s_flxs, s_fpts)
}})
s_points.keys()

In [ ]:
save_path = "../result/S001/spect.hdf5"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with h5.File(save_path, "w") as f:
    for path, val in s_points.items():
        grp = f.create_group(path)
        grp.create_dataset("cflx", data=val["cflx"])
        spect_grp = grp.create_group("spectrum")
        spect_grp.create_dataset("flxs", data=val["spectrum"][0])
        spect_grp.create_dataset("fpts", data=val["spectrum"][1])
        spect_grp.create_dataset("data", data=val["spectrum"][2])
        points_grp = grp.create_group("points")
        points_grp.create_dataset("flxs", data=val["points"][0])
        points_grp.create_dataset("fpts", data=val["points"][1])

In [ ]:
load_path = "../result/S001/spect.hdf5"
s_points = {}
with h5.File(load_path, "r") as f:
    for key in f.keys():
        grp = f[key]
        cflx = grp["cflx"][()]
        spectrum = (
            grp["spectrum"]["flxs"][()],
            grp["spectrum"]["fpts"][()],
            grp["spectrum"]["data"][()]
        )
        points = (
            grp["points"]["flxs"][()],
            grp["points"]["fpts"][()]
        )
        s_points.update({key: {
            "cflx": cflx,
            "spectrum": spectrum,
            "points": points
        }})
s_points.keys()

# Align cflx

In [ ]:
target_cflx = list(s_points.values())[-1]["cflx"]
for spect in s_points.values():
    shift = target_cflx - spect["cflx"]
    spect["cflx"] += shift
    spect["spectrum"] = (spect["spectrum"][0] + shift, *spect["spectrum"][1:])
    spect["points"] = (spect["points"][0] + shift, *spect["points"][1:])

# Search in Database

In [ ]:
s_flxs = np.concatenate([s_points[k]["points"][0] for k in s_points.keys()])
s_fpts = np.concatenate([s_points[k]["points"][1] for k in s_points.keys()])
flx_bound = (
    np.nanmin([np.nanmin(s_points[k]["spectrum"][0]) for k in s_points.keys()]),
    np.nanmax([np.nanmax(s_points[k]["spectrum"][0]) for k in s_points.keys()])
)
flxs = np.linspace(flx_bound[0], flx_bound[1], 500)
fpt_bound = (
    np.nanmin([np.nanmin(s_points[k]["spectrum"][1]) for k in s_points.keys()]),
    np.nanmax([np.nanmax(s_points[k]["spectrum"][1]) for k in s_points.keys()])
)
fpts = np.linspace(fpt_bound[0], fpt_bound[1], 500)

In [ ]:
allows = {
    "transitions": [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)],
    # "blue side": [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)],
    # "red side": [(0, 3)],
    # "mirror": [(0, 1), (0, 2), (1, 3)],
    # "transitions2": [(0, 3), (1, 2), (1, 3)],
    # "blue side2": [(1, 2)],
    # "red side2": [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)],
    # "mirror2": [(0, 3)],
    "r_f": 7.4876,
    # "sample_f": 6.88128,
}

In [ ]:
best_params, _, _ = search_in_database(
    s_flxs, s_fpts, "../database/fluxonium_1.h5", allows
)
best_params

In [ ]:
f_energies = calculate_energy(flxs, *best_params, cutoff=40)

In [ ]:
fig = go.Figure()

for spect in s_points.values():
    # 取得對應數據和範圍
    data = spect["spectrum"][2]
    flx_mask = np.any(~np.isnan(data), axis=0)
    fpt_mask = np.any(~np.isnan(data), axis=1)
    data = data[fpt_mask, :][:, flx_mask]
    data = np.abs(data - np.mean(data, axis=0, keepdims=True))
    data /= np.std(data, axis=0, keepdims=True)

    # 使用 imshow 繪製，指定範圍
    fig.add_trace(
        go.Heatmap(
            z=data,
            x=spect["spectrum"][0][flx_mask],
            y=spect["spectrum"][1][fpt_mask],
            colorscale="Viridis",
            showscale=False,
        )
    )

s_allows = allows
fs, _, labels = energy2transition(f_energies, s_allows)
for i, label in enumerate(labels):
    fig.add_trace(go.Scatter(x=flxs, y=fs[:, i], mode="lines", name=label))

fig.add_trace(go.Scatter(x=s_flxs, y=s_fpts, mode="markers", marker=dict(color="red", size=3)))

fig.update_layout(
    xaxis=dict(title="Flux"),
    yaxis=dict(title="Frequency"),
    legend=dict(x=1, y=0.5),
)
# set x and y axis range
fig.update_xaxes(range=[flx_bound[0], flx_bound[1]])
fig.update_yaxes(range=[fpt_bound[0], fpt_bound[1]])

fig.show()

# Scipy

In [ ]:
EJb = (2.0, 10.0)
ECb = (0.5, 3.0)
ELb = (0.1, 2.0)
# EJb = (3.0,8.0)
# ECb = (0.5, 2.5)
# ELb = (0.1, 1.5)

# allows = {
#     "transitions": [(0, 1), (2, 3)],
#     # "transitions": [(0, 1)],
#     "mirror": [],
#     "sample_f": 6.88128,
# }

In [ ]:
# fit the spectrumData
sp_params = fit_spectrum(s_flxs, s_fpts, best_params, allows, (EJb, ECb, ELb))
# sp_params = fit_spectrum(s_flxs, s_fpts, sp_params, allows, (EJb, ECb, ELb))

# print the results
print("Fitted params:", *sp_params)

In [ ]:
# sp_params = (8.51795441, 0.9041685, 1.09072694)
f_energies = calculate_energy(flxs, *sp_params, 40)
# f_energies = calculate_energy(flxs, 8.51, 0.904, 1.09, 60)

In [ ]:
fig = go.Figure()

for spect in s_points.values():
    # 取得對應數據和範圍
    data = spect["spectrum"][2]
    flx_mask = np.any(~np.isnan(data), axis=0)
    fpt_mask = np.any(~np.isnan(data), axis=1)
    data = data[fpt_mask, :][:, flx_mask]
    data = np.abs(data - np.mean(data, axis=0, keepdims=True))
    data /= np.std(data, axis=0, keepdims=True)

    # 使用 imshow 繪製，指定範圍
    fig.add_trace(
        go.Heatmap(
            z=data,
            x=spect["spectrum"][0][flx_mask],
            y=spect["spectrum"][1][fpt_mask],
            colorscale="Viridis",
            showscale=False,
        )
    )

s_allows = allows
fs, _, labels = energy2transition(f_energies, s_allows)
for i, label in enumerate(labels):
    fig.add_trace(go.Scatter(x=flxs, y=fs[:, i], mode="lines", name=label))

fig.add_trace(go.Scatter(x=s_flxs, y=s_fpts, mode="markers", marker=dict(color="red", size=3)))

fig.update_layout(
    title=f"EJ/EC/EL = ({sp_params[0]:.2f}, {sp_params[1]:.2f}, {sp_params[2]:.2f})",
    title_x=0.5,
    xaxis=dict(title="Flux"),
    yaxis=dict(title="Frequency"),
    legend=dict(x=1, y=0.5),
)

# set x and y axis range
fig.update_xaxes(range=[flx_bound[0], flx_bound[1]])
fig.update_yaxes(range=[fpt_bound[0], fpt_bound[1]])

fig.show()

In [ ]:
# dump the data
savepath = "../result/S001/params.csv"

df = pd.DataFrame(
    {
        "Qubit": [qub_name],
        "Half flux": [cflx],
        "Period": [2*(eflx - cflx)],
        "EJ": [sp_params[0]],
        "EC": [sp_params[1]],
        "EL": [sp_params[2]],
    }
)

if os.path.exists(savepath):
    old = pd.read_csv(savepath)
    old = old[old["Qubit"] != qub_name]
    df = pd.concat([old, df], ignore_index=True)
df.to_csv(savepath, header=True, index=False)